# **Detection of hate speech in Arabic tweets-phase one**



In [1]:
!pip install pyarabic #A specific Arabic language library for Python, provides basic functions to manipulate Arabic letters and text, like detecting Arabic letters, Arabic letters groups and characteristics, remove diacritics etc.
import pyarabic.araby as araby
import re
import string #for string.punctuation


     |████████████████████████████████| 112kB 5.4MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=7df8f371ddee0291ab8f3b23e480a0cbf21f544572e8266a1f958149ddc4a4c0
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic


In [2]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, naive_bayes, svm
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report as creport
from multiprocessing import Pool


In [3]:

import pandas as pd
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")

In [4]:
dfTrain

,Unnamed: 0,Text,Offensive,Hate
0,0,الحمدلله يارب فوز مهم يا زمالك.. كل الدعم ليكم...,NOT_OFF,NOT_HS
1,1,فدوه يا بخت فدوه يا زمن واحد منكم يجيبه,NOT_OFF,NOT_HS
2,2,RT @USER: يا رب يا واحد يا أحد بحق يوم الاحد ا...,OFF,HS
3,3,RT @USER: #هوا_الحرية يا وجع قلبي عليكي يا امي...,NOT_OFF,NOT_HS
4,4,يا بكون بحياتك الأهم يا إما ما بدي أكون 🎼,NOT_OFF,NOT_HS
...,...,...,...,...
6834,6834,@USER يا حمار ، يا جاهل ، نسبة الباطل ما بتتحس...,OFF,NOT_HS
6835,6835,RT @USER: @USER كل زق يا طاقية يا واطي يا حقير...,OFF,NOT_HS
6836,6836,@USER<LF>يا كبير يا ممتع يا نجم لابد أن تعي جي...,NOT_OFF,NOT_HS
6837,6837,يا رب الاتحاد يفوز يا رب. 😭😭 #الاتحاد_النصر,NOT_OFF,NOT_HS


# **nulls check**

In [5]:
print("Train data shape:{} \nDev data shape: {}".format(dfTrain.shape,dfTest.shape))


Train data shape:(6839, 4) 
Dev data shape: (1000, 4)


In [6]:
dfTrain.isnull().sum()

Unnamed: 0    0
Text          0
Offensive     0
Hate          0
dtype: int64

In [7]:
dfTest.isnull().sum()

Unnamed: 0    0
Text          0
Offensive     0
Hate          0
dtype: int64

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list=stopwords.words("arabic")
print(stopwords_list)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على'

# **Text Cleaning and Preprocessing**

In [9]:
arabic_punctuations = '''`÷« »×؛<>٩٨'٧٦٥٤٣٢١٠_()↗*•&^%][ـ،/:"؟.,'{}⋮≈~¦+|٪!”…“–ـ/[]%=#*+\\•~@£·_{}©^®`→°€™›♥←×§″′Â█à…“★”–●â►−¢¬░¶↑±▾	═¦║―¥▓—‹─▒：⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡₹´'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


In [10]:
emoji_pattern = re.compile("["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+" , flags=re.UNICODE)

In [11]:
def clean(text):
  text= re.sub("@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]", "", text)
  text = re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", text)
  text = re.sub(arabic_diacritics, '', text)
  #remove emoji
  text = re.sub(emoji_pattern, "", text)
  text = re.sub('emoji[0-9][0-9][0-9]', '', text) #remove emoji encodings
 
  return text





def remove_diacritics(text):
  #text = str(text).apply(lambda x: _remove_diacritics(x))
  #return text
  text = str(text)
  text= re.sub(arabic_diacritics, '', text)
  return text

def remove_punctuations(x):
  x = str(x)
  #translator = str.maketrans(' ', ' ', punctuations_list)
  translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
  return x.translate(translator)
    

def remove_repeating_char(x):
  x = str(x)
  return re.sub(r'(.)\1+', r'\1', x)


  
def remove_english_word_and_numbers(x):
  x = str(x)
  return re.sub(r'[a-zA-Z0-9]+', '', x)


def clean_space(x):
  compiled_re = re.compile(r"\s+")
  return compiled_re.sub(" ", x)

   

In [12]:
exclude = set(string.punctuation+'؟،٪؛') #string.punctuation will give all sets of punctuation.


In [13]:
def remove_tatweel(text):
    text = re.sub("[«»،؟,ـ]", "", text)
    text=re.sub(r'([أبتثجحخدذرزسشصضطظعغفقكلمنهويىآؤإاءئ])\1+', r'\1', text) 
    #text = re.sub("ـ", "", text)
    #text = re.sub("«", "", text)
    #text = re.sub("»", "", text)
    #text = re.sub("،", "", text)
    #text = re.sub("»", "", text)
    #text = re.sub(",", "", text)
    #text = re.sub("؟", "", text)
    return(text)

# Normalize

In [14]:
def normalize_arabic(x):
  x = re.sub("[إأآا]", "ا", x)
  x = re.sub("ى", "ي", x)
  x = re.sub("ؤ", "ء", x)
  x = re.sub("ئ", "ء", x)
  x = re.sub("ة", "ه", x)
  x = re.sub("گ", "ك", x)
  return x

In [15]:
for x in range(len(dfTrain)):
  newtweet=""
  text = dfTrain["Text"][x]
  text=text.split()
  text_without_sw = [word for word in text if not word in stopwords_list]
  for txt in text_without_sw:
    txt= clean(txt)
    txt= remove_diacritics(txt)
    txt= remove_punctuations(txt)
    txt= remove_repeating_char(txt)
    txt= remove_english_word_and_numbers(txt)
    txt= clean_space(txt)
    txt= normalize_arabic(txt)
    

    txt= araby.strip_tashkeel(txt)
    if txt.endswith('#') or txt.startswith('#') == False:
      if re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", txt) == None: 
        txt= remove_tatweel(txt)
        txt= ''.join(ch for ch in txt if ch not in exclude) #Join all items in a tuple into a string
      newtweet=newtweet+txt+" "
  dfTrain["Text"][x] = newtweet
  #print(tweet)
  dfTrain = dfTrain.replace('\n','', regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# **Feature Engineering**

In [34]:
from google.colab import files

In [ ]:

data_to_load = files.upload()

Saving done.xlsx to done.xlsx


In [16]:
dfword=pd.read_excel("done.xlsx") #list of hate word

In [18]:
dfword.isnull().sum()

term    0
dtype: int64

In [19]:
dfTrain["NHW"]=""

In [20]:
for index, row in dfTrain.iterrows():
  text=row['Text'].split()
  n=0

  for i in dfword['term']:
    if i in text:
     n=n+1
  dfTrain['NHW'][index]=n


   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
for x in range(len(dfTest)):
  newtweet=""
  text = dfTest["Text"][x]
  text=text.split()
  text_without_sw = [word for word in text if not word in stopwords_list]
  for txt in text_without_sw:
    txt= clean(txt)
    txt= remove_diacritics(txt)
    txt= remove_punctuations(txt)
    txt= remove_repeating_char(txt)
    txt= remove_english_word_and_numbers(txt)
    txt= clean_space(txt)
    txt= normalize_arabic(txt)

    txt= araby.strip_tashkeel(txt)
    if txt.endswith('#') or txt.startswith('#') == False:
      if re.match("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", txt) == None: 
        txt= remove_tatweel(txt)
        txt= ''.join(ch for ch in txt if ch not in exclude) #Join all items in a tuple into a string
      newtweet=newtweet+txt+" "
  dfTest["Text"][x] = newtweet
  #print(tweet)
  dfTest = dfTest.replace('\n','', regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
AD  = {"NOT_HS": 0, "HS": 1}
data = [dfTrain, dfTest]

for dataset in data:
    dataset['Hate'] = dataset['Hate'].map(AD)

AD1 = {"NOT_OFF": 0, "OFF": 1}
data = [dfTrain, dfTest]

for dataset in data:
    dataset['Offensive'] = dataset['Offensive'].map(AD1)

In [23]:
dfTest


,Unnamed: 0,Text,Offensive,Hate
0,0,في حاجات مينفعش نلفت نظركوا ليها زي الاصول كده...,0,0
1,1,وعيون تنادينا تحايل فينا و نقول عيونه اما ب...,0,0
2,2,بلادي البلاد بلادي بحبك مصر بحبك بلادي بحبك بح...,0,0
3,3,رب قوي معين مدني بالقوه و العون و الصبر انت...,0,0
4,4,رحمك اله صدام بطل ومقدام,0,0
...,...,...,...,...
995,995,انتو بتوزعوا زيت وسكر فعلا عباس يا عباس انا...,0,0
996,996,كدا عمر متزعلهاش حبيبي,0,0
997,997,هدا سكن اطفال امارتين شارقه طالبين فزعتكم اهل ...,0,0
998,998,ومدني بمد قوتك اواجه ضعفي وكن ربي احتاجك ...,0,0


In [24]:
dfTrain.head(5)

,Unnamed: 0,Text,Offensive,Hate,NHW
0,0,الحمدله يارب فوز مهم زمالك الدعم ليكم رجاله ...,0,0,0
1,1,فدوه بخت فدوه زمن واحد منكم يجيبه,0,0,0
2,2,رب واحد احد بحق يوم الاحد ان تهلك بني سعود ...,1,1,0
3,3,هوا الحريه وجع قلبي عليكي امي اله يحرق قلب...,0,0,0
4,4,بكون بحياتك الاهم بدي اكون,0,0,0


# **CAMeL**


In [25]:
%pip install camel-tools

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 778kB 9.8MB/s 
     |████████████████████████████████| 256kB 39.5MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
     |████████████████████████████████| 3.0MB 37.3MB/s 
     |████████████████████████████████| 1.2MB 37.2MB/s 
  Created wheel for camel-tools: filename=camel_tools-1.1.0-cp37-none-any.whl size=96899 sha256=ec637dd5bb1f7ba08e1578f5ca549ff8429394e99b3f102a84b108e4d179a493
  Stored in directory: /root/.cache/pip/wheels/ff/8d/46/949198daef42f9e3bf64ed645a4af05816f47aa7677ac74589
  Created wheel for camel-kenlm: filename=camel_kenlm-2020.11.2-cp37-cp37m-linux_x86_64.whl size=2313808 sha256=f40206e47d4d909d838066c186dd58cd25cf3d41652f24199b4d218e673e7659
  Stored in directory: /root/.cache/pip/wheels/a4/df/36/46723b5cb1a11c345ad10813f13d2ce2452e95b0aaca7ad332
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=59c63f320fcde5d9d2a

In [26]:
#Mount drive
from google.colab import drive
import os

drive.mount('/gdrive')

%mkdir /gdrive/MyDrive/camel_tools

Mounted at /gdrive
mkdir: cannot create directory ‘/gdrive/MyDrive/camel_tools’: File exists


In [27]:
#Install the data
os.environ['CAMELTOOLS_DATA'] = '/gdrive/MyDrive/camel_tools'

!export | camel_data full #light or or full

In [28]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

In [29]:

dfTrain['Text2']="a"


mle = MLEDisambiguator.pretrained('calima-msa-r13')

for index, row in dfTrain.iterrows():
 
  sentence = simple_word_tokenize(row['Text'])


  tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
  tokens = tokenizer.tokenize(sentence)
  dfTrain['Text2'][index]=tokens
  print(tokens)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Streaming output truncated to the last 5000 lines.
['ناس', '+ي', 'و', 'أغلى', '+هم']
['تعب', 'مليونلكن', 'أنا', 'أنتي', 'حمار', '+ه']
['يلا', 'يحي', 'يلا', 'يحي', 'مود', 'اكتفيتد']
['ال+', 'سلام', 'على', '+كم', 'ال+', 'هم', 'أن', '+ني', 'أسأل', '+ك', 'ب+', 'رحمة', '+ك', 'ياحي', 'ياقيوم', 'ياحي', 'ياقيوم', 'ياحي', 'ياقيوم', 'عبد', '+ك', 'وهاذا', 'أمام', '+ك', 'هوا', 'عبد', '+ك', 'أل', '+ه', 'يدي', '+ك', 'ال+', 'شافي', 'ال+', 'معافى', 'اشفء', 'يأل', '+ه', 'يأل', '+ه', 'يأل', '+ه', 'ب+', 'رحمة', '+ك', 'و+', 'عفو', '+ك', 'و+', 'رضا', '+ك', 'و+', 'ل', '+ك', 'ال+', 'فضل', 'و+', 'ألمن', '+ه', 'و+', 'أنت', 'أهل', 'هاذا', 'ال+', 'فضل', 'ال+', 'عظيم', 'عظيم']
['جدعان', 'ريت', 'إلى', 'يشوف', 'ال+', 'حلقة', 'ميحقش', 'حاجة']
['عمر', '+ي', 'جلاد', 'عمر', '+ي', 'موسى']
['أنا', 'أضحك', 'أتكلم']
['تعال', 'حسينوه', 'تعال', 'ل+', 'عم', '+ك']
['أنا', 'و+', 'من', '+ه', 'كنا', 'ب+', 'نمش', '+ي', 'ف', 'ال+', 'شارع', 'نغني', '+ها', 'سوأ', 'أيام', 'شمس', 'منورة', 'غيب', '+ي', 'و+', 'كفاية', 'ضيك', 'حبيبي', 'ود

In [30]:
dfTest['Text2']="a"


mle = MLEDisambiguator.pretrained('calima-msa-r13')

for index, row in dfTest.iterrows():
  text=row['Text']
  sentence = simple_word_tokenize(text)


  tokenizer = MorphologicalTokenizer(mle, scheme='d3tok', split=True)
  tokens = tokenizer.tokenize(sentence)
  dfTest['Text2'][index]=tokens
  print(tokens)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


['في', 'حاجات', 'مينفعش', 'نلفت', 'نظركوا', 'لي', '+ها', 'زي', 'ال+', 'أصول', 'كد', '+ه', 'اتربيتوا', 'على', '+ها', 'لا']
['و+', 'عيون', 'تنادينا', 'تحايل', 'في', '+نا', 'و', 'نقول', 'عيون', '+ه', 'أما', 'بلاش']
['بلاد', '+ي', 'ال+', 'بلاد', 'بلاد', '+ي', 'ب+', 'حب', '+ك', 'مصر', 'ب+', 'حب', '+ك', 'بلاد', '+ي', 'ب+', 'حب', '+ك', 'ب+', 'حب', '+ك', 'مصر']
['رب', 'قوي', 'معين', 'مدني', 'ب+', 'ال+', 'قوة', 'و', 'ال+', 'عون', 'و', 'ال+', 'صبر', 'انتهت', 'طاقتي', 'و', 'نفذ', 'صبري', 'ف+', 'صبر', '+ني']
['رحم', '+ك', 'أل', '+ه', 'صدام', 'بطل', 'و+', 'مقدام']
['جابر', 'بنجرجي', 'مرتزق', 'حثالة', 'ال+', 'حمدين', 'خربوط', 'ساذج', 'مليق', 'إمع', '+ه', 'مسيلمه', 'ال+', 'كذاب', 'هامشهري', 'آغا', 'ملال', '+ي', 'إيراني', 'زهرا', 'مار', 'يا', 'تشمت', 'ب+', 'مصر', 'و+', 'أبناء', 'مصر', 'أتعلم', 'ب+', 'أن', '+ك', 'ب+', 'رجل', 'يوصف', 'ك+', 'رجل', 'اتباع', 'ال+', 'رجال', 'أنت', 'أنت', 'حثالة', 'اتباع', 'ال+', 'حثالة', 'خسيس']
['أنت', 'بعض', '+ي', 'و+', 'كلي', 'و+', 'ذات', '+ي', 'يا', 'لون', 'ال+', 'فرح',

In [ ]:
dfTrain

,Unnamed: 0,Text,Offensive,Hate,NHW,Text2
0,0,الحمدله يارب فوز مهم زمالك الدعم ليكم رجاله ...,NOT_OFF,0,0,"[ال+, حمدلة, يأرب, فوز, مهم, زمالك, ال+, دعم, ..."
1,1,فدوه بخت فدوه زمن واحد منكم يجيبه,NOT_OFF,0,0,"[فدوا, +ه, بخت, فدوا, +ه, زمن, واحد, من, +كم, ..."
2,2,رب واحد احد بحق يوم الاحد ان تهلك بني سعود ...,OFF,1,0,"[رب, واحد, أحد, ب+, حق, يوم, ال+, أحد, أن, تهل..."
3,3,هوا الحريه وجع قلبي عليكي امي اله يحرق قلب...,NOT_OFF,0,0,"[هوا, ال+, حرية, وجع, قلب, +ي, عليكي, أم, +ي, ..."
4,4,بكون بحياتك الاهم بدي اكون,NOT_OFF,0,0,"[ب+, كون, ب+, حياة, +ك, ال+, أهم, بد, +ي, أكون]"
...,...,...,...,...,...,...
6834,6834,حمار جاهل نسبه الباطل بتحسبش و عاملي ن...,OFF,0,6,"[حمار, جاهل, نسب, +ه, ال+, باطل, بتحسبش, و, عا..."
6835,6835,زق طاقيه واطي حقير تافه سافل اتفو,OFF,0,0,"[زق, طاقية, واطي, حقير, تافه, سافل, اتفو]"
6836,6836,يا كبير متع نجم لابد تعي جيدا انا وصلنا لهذا ...,NOT_OFF,0,0,"[يا, كبير, متع, نجم, ل+, آبد, تعي, جيدا, أنا, ..."
6837,6837,رب الاتحاد يفوز رب الاتحاد النصر,NOT_OFF,0,0,"[رب, ال+, اتحاد, يفوز, رب, ال+, اتحاد, ال+, نصر]"


In [ ]:
dfTest


,Unnamed: 0,Text,Offensive,Hate,Text2
0,0,في حاجات مينفعش نلفت نظركوا ليها زي الاصول كده...,NOT_OFF,0,"[في, حاجات, مينفعش, نلفت, نظركوا, لي, +ها, زي,..."
1,1,وعيون تنادينا تحايل فينا و نقول عيونه اما ب...,NOT_OFF,0,"[و+, عيون, تنادينا, تحايل, في, +نا, و, نقول, ع..."
2,2,بلادي البلاد بلادي بحبك مصر بحبك بلادي بحبك بح...,NOT_OFF,0,"[بلاد, +ي, ال+, بلاد, بلاد, +ي, ب+, حب, +ك, مص..."
3,3,رب قوي معين مدني بالقوه و العون و الصبر انت...,NOT_OFF,0,"[رب, قوي, معين, مدني, ب+, ال+, قوة, و, ال+, عو..."
4,4,رحمك اله صدام بطل ومقدام,NOT_OFF,0,"[رحم, +ك, أل, +ه, صدام, بطل, و+, مقدام]"
...,...,...,...,...,...
995,995,انتو بتوزعوا زيت وسكر فعلا عباس يا عباس انا...,NOT_OFF,0,"[أنتو, بتوزعوا, زيت, و+, سكر, فعلا, عباس, يا, ..."
996,996,كدا عمر متزعلهاش حبيبي,NOT_OFF,0,"[كدا, عمر, متزعلهاش, حبيبي]"
997,997,هدا سكن اطفال امارتين شارقه طالبين فزعتكم اهل ...,NOT_OFF,0,"[هدآ, سكن, أطفال, إمارتين, شارقة, طالبين, فزعة..."
998,998,ومدني بمد قوتك اواجه ضعفي وكن ربي احتاجك ...,NOT_OFF,0,"[و+, مدن, +ي, ب+, مد, قوة, +ك, أواجه, ضعف, +ي,..."


In [35]:
dfTest.to_excel("tokeTest.xlsx")  
files.download('tokeTest.xlsx')

dfTrain.to_excel("tokeTrain.xlsx")  
files.download('tokeTrain.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Variables and predictors

In [36]:
pip install sklearn-pandas


In [37]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn_pandas import DataFrameMapper
from sklearn.svm import LinearSVC

In [38]:
y = dfTrain["Hate"]
y.value_counts()

0    6489
1     350
Name: Hate, dtype: int64

In [39]:
le = preprocessing.LabelEncoder()
y_enc = le.fit_transform(y)

In [40]:

vect = TfidfVectorizer()
X_train= dfTrain["Text2"].apply(lambda x: ' '.join(x))
Y_train = dfTrain["Hate"]
X_test= dfTest["Text2"].apply(lambda x: ' '.join(x))
Y_test = dfTest["Hate"]
vect.fit(X_train)
X_train_df = vect.transform(X_train)
X_test_df = vect.transform(X_test)

# **Hate Speech Detection Using Logistic Regression and Tf-idf**

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [43]:
LRmodel = LogisticRegression()
LRmodel.fit(X_train_df,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
# Making predictions

prediction = dict()
prediction["Logistic"] = LRmodel.predict(X_test_df)
print(prediction["Logistic"])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [45]:
# Reviewing the metrics
print ("Accuracy:",accuracy_score(Y_test,prediction["Logistic"]))

Accuracy: 0.956


In [46]:
print(classification_report(Y_test,prediction["Logistic"]))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98       956
           1       0.00      0.00      0.00        44

    accuracy                           0.96      1000
   macro avg       0.48      0.50      0.49      1000
weighted avg       0.91      0.96      0.93      1000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Hate Speech Detection Using Logistic Regression and Tf-idf + Other Features**

In [67]:
data = pd.concat([dfTrain, dfTest], ignore_index=True)

In [88]:
data['Text2']=data['Text2'].values.astype('U')
mapper = DataFrameMapper([
     ("Text2", TfidfVectorizer()),
     ("Offensive", None),
   

     
 ])


In [90]:
features= mapper.fit_transform(data)
# Split the data between train and test
x, x_test, y, y_test = train_test_split(features,data['Hate'],test_size=0.2,train_size=0.8, random_state = 10)

LRmodel2 = LogisticRegression()
LRmodel2.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [91]:
prediction = dict()

prediction["Logistic2"] = LRmodel2.predict(x_test)
print(prediction["Logistic2"])
print ("Accuracy:",accuracy_score(y_test,prediction["Logistic2"]))
print(classification_report(y_test,prediction["Logistic2"]))

[0 0 0 ... 0 0 0]
Accuracy: 0.9572704081632653
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1499
           1       1.00      0.03      0.06        69

    accuracy                           0.96      1568
   macro avg       0.98      0.51      0.52      1568
weighted avg       0.96      0.96      0.94      1568



# **Hate Speech Detection Using Support Vector Machine (SVM) and Tf-idf**

In [72]:
svm = LinearSVC().fit(X_train_df,Y_train)

In [73]:
prediction["SVM"] = svm.predict(X_test_df)
print(prediction["SVM"])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [74]:
accuracy_score(Y_test,prediction["SVM"])

0.96

In [75]:
print(classification_report(Y_test,prediction["SVM"]))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       956
           1       0.70      0.16      0.26        44

    accuracy                           0.96      1000
   macro avg       0.83      0.58      0.62      1000
weighted avg       0.95      0.96      0.95      1000



# **Emotion Detection Using Support Vector Machine (SVM) and Tf-idf + Other Features**

In [93]:
svm2 = LinearSVC().fit(x,y)

In [94]:
prediction["SVM2"] = svm2.predict(x_test)
print(prediction["SVM2"])
print ("Accuracy:",accuracy_score(y_test,prediction["SVM2"]))
print(classification_report(y_test,prediction["SVM2"]))

[0 0 0 ... 0 0 0]
Accuracy: 0.9630102040816326
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1499
           1       0.70      0.28      0.40        69

    accuracy                           0.96      1568
   macro avg       0.84      0.64      0.69      1568
weighted avg       0.96      0.96      0.96      1568



# **Aravec**


In [95]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [96]:
import gensim
import re
import spacy

In [98]:
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt 

import time as time 



In [99]:
import gensim
import re
import numpy as np
from nltk import ngrams



In [100]:
# load the AraVec model
model =gensim.models.Word2Vec.load("full_uni_cbow_100_wiki.mdl")

print("We've",len(model.wv.index2word),"vocabularies")

We've 320636 vocabularies


In [101]:
# make a directory called "spacyModel"
%mkdir spacyModel

In [102]:
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

In [103]:
!gzip ./spacyModel/aravec.txt

In [104]:
!python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz

✔ Successfully created model
320636it [00:14, 21669.87it/s]
✔ Loaded vectors from spacyModel/aravec.txt.gz
✔ Sucessfully compiled vocab
320826 entries, 320636 vectors


In [105]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [106]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [107]:
#Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

In [108]:
# Test your model
nlp("حمار").vector

array([-1.4749993 ,  0.22775699,  1.5759528 , -0.28790772, -2.6447978 ,
       -1.0114739 , -1.1197586 , -0.3060226 , -1.557654  ,  0.39123395,
       -1.0945843 ,  0.1327005 ,  1.9071548 ,  0.38472483, -1.4417952 ,
       -0.2537513 ,  0.90716004, -1.2737074 ,  0.13072449,  0.0103218 ,
       -1.4402106 ,  1.8464572 , -0.47556457,  1.0440905 , -0.01697282,
       -1.7750589 ,  0.6115579 , -1.3845533 ,  2.1270971 , -1.004463  ,
        0.95853776,  0.5524393 , -2.0738297 , -1.2363503 ,  0.8661258 ,
        0.33865142, -1.0555165 , -1.9049265 ,  0.950684  ,  0.67392623,
       -0.75993305,  1.291433  ,  1.2853988 ,  0.4452732 , -1.107891  ,
        0.91399646, -1.7445751 , -0.9924268 ,  0.6019321 ,  1.0266421 ,
       -0.7916266 , -0.49058706,  1.9360733 , -1.4314659 , -0.8985691 ,
       -1.2918841 , -2.3527465 ,  3.4417334 , -0.05990298,  0.541461  ,
        0.52952814,  1.723453  , -1.0476257 ,  0.35789263, -0.02894059,
        0.66261595,  2.3797858 ,  1.2898816 , -1.670391  , -1.04

# **hate speech detection Using Logistic Regression and Aravec**

In [109]:
import csv
from io import open
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.svm import LinearSVC

In [110]:
df = pd.read_excel("tokeTrain.xlsx")
print (df['Text'])
test = pd.read_excel("tokeTest.xlsx")
print (test['Text'])
# print the number of data
print(len(df))
print(len(test))

0       الحمدله يارب فوز مهم زمالك  الدعم ليكم رجاله  ...
1                      فدوه بخت فدوه زمن واحد منكم يجيبه 
2          رب واحد احد بحق يوم الاحد ان تهلك بني سعود ...
3           هوا الحريه وجع قلبي عليكي امي اله يحرق قلب...
4                            بكون بحياتك الاهم بدي اكون  
                              ...                        
6834      حمار   جاهل   نسبه الباطل بتحسبش   و عاملي ن...
6835                   زق طاقيه واطي حقير تافه سافل اتفو 
6836     يا كبير متع نجم لابد تعي جيدا انا وصلنا لهذا ...
6837                 رب الاتحاد يفوز رب    الاتحاد النصر 
6838    يعز عليا ادخل الشارع وملاقيكش واقف مستنيني الب...
Name: Text, Length: 6839, dtype: object
0      في حاجات مينفعش نلفت نظركوا ليها زي الاصول كده...
1         وعيون تنادينا تحايل فينا و نقول عيونه اما ب...
2      بلادي البلاد بلادي بحبك مصر بحبك بلادي بحبك بح...
3         رب قوي معين مدني بالقوه و العون و الصبر انت...
4                            رحمك اله صدام بطل ومقدام   
                             ...     

In [111]:
x = np.array([nlp(d).vector for d in df['Text2']])
x2 = np.array([nlp(d).vector for d in test['Text2']])


y = np.array([d for d in df['Hate']])
y2 = np.array([d for d in test['Hate']])

X_train_df1 = x
Y_train1 = y
X_test_df1 = x2
Y_test1 = y2

In [112]:
logreg = linear_model.LogisticRegression()
logreg.fit(X_train_df1,Y_train1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [113]:
prediction = dict()
prediction["lr"] = logreg.predict(X_test_df1)

In [114]:
print(classification_report(Y_test1,prediction["lr"]))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       956
           1       0.57      0.09      0.16        44

    accuracy                           0.96      1000
   macro avg       0.77      0.54      0.57      1000
weighted avg       0.94      0.96      0.94      1000



# **Hate speech Detection Using Logistic Regression and Aravec + Other Features**

In [138]:
features2=df.loc[:, ["Offensive"]].values
tweets= np.concatenate([x, features2], axis=1)

In [139]:
X_train_tweets = tweets

In [140]:
features2test=test.loc[:,  ["Offensive"]].values
tweets2= np.concatenate([x2, features2test], axis=1)
X_test_tweets = tweets2

In [141]:
logreg2 = linear_model.LogisticRegression()
logreg2.fit(X_train_tweets,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [142]:
prediction = dict()
prediction["lr2"] = logreg2.predict(X_test_tweets)
print(prediction["lr2"])

[0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [143]:
print(classification_report(Y_test,prediction["lr2"]))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       956
           1       0.71      0.34      0.46        44

    accuracy                           0.96      1000
   macro avg       0.84      0.67      0.72      1000
weighted avg       0.96      0.96      0.96      1000



# **Hate speech Detection Using SVM and Aravec + Other Features**

In [144]:
svm2 = LinearSVC().fit(X_train_tweets,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [145]:
prediction = dict()
prediction["svm2"] = svm2.predict(X_test_tweets)
print(prediction["svm2"])

[0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [146]:
print(classification_report(Y_test,prediction["svm2"]))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       956
           1       0.74      0.39      0.51        44

    accuracy                           0.97      1000
   macro avg       0.86      0.69      0.75      1000
weighted avg       0.96      0.97      0.96      1000

